In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import time
import datetime
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv', index_col='id')
print(data['sales'].min(), data['sales'].max())

In [ ]:
data['date'] = pd.to_datetime(data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.dayofweek
data['year'] = data['date'].dt.year

test_data['month'] = test_data['date'].dt.month
test_data['day'] = test_data['date'].dt.dayofweek
test_data['year'] = test_data['date'].dt.year

In [ ]:
x_columns = ['store', 'item', 'month', 'day', 'year']
X = data[x_columns]
Y = data.iloc[:,3]

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=.2)

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=1000, max_depth = 6, eta=.025, eval_metric='mae')
t0 = time.time()
model.fit(train_X, train_Y, early_stopping_rounds=20, eval_set=[(test_X, test_Y)], verbose=False)
print('Runtime: ' + str(round(time.time() - t0, 3)) + 's')
predictions = model.predict(test_X)
print(mean_absolute_error(test_Y, predictions))
print(model)
submissions = model.predict(test_data[x_columns])
test_data['sales'] = submissions

In [ ]:
sub_columns = ['sales']
test_data[sub_columns].to_csv('predictions_submission.csv', index=True)